## PT DDP Launcher Testing
This notebook tests the following combination:

* image: PT training DLC with my changes
* distribution = pytorchddp, backend = nccl

In [20]:
#!pip uninstall -y sagemaker

In [24]:
#Upload sagemaker-python-sdk whl from local machine and install it here
%pip install sagemaker-2.94.0.dev0-py2.py3-none-any.whl

Processing ./sagemaker-2.94.0.dev0-py2.py3-none-any.whl
  Using cached importlib_metadata-1.7.0-py2.py3-none-any.whl (31 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.7.0
    Uninstalling importlib-metadata-3.7.0:
      Successfully uninstalled importlib-metadata-3.7.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip show sagemaker

Name: sagemaker
Version: 2.94.0.dev0
Summary: Open source library for training and deploying models on Amazon SageMaker.
Home-page: https://github.com/aws/sagemaker-python-sdk/
Author: Amazon Web Services
Author-email: 
License: Apache License 2.0
Location: /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages
Requires: attrs, boto3, google-pasta, importlib-metadata, numpy, packaging, pandas, pathos, protobuf, protobuf3-to-dict, smdebug-rulesconfig
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()


print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
#Add instructions for local environment later, if needed

sagemaker role arn: arn:aws:iam::570106654206:role/Dev
sagemaker bucket: sagemaker-us-west-2-570106654206
sagemaker session region: us-west-2


In [3]:
region = "us-west-2"
image = (
    "pt-ddp-custom"  # Example: pt-smdataparallel-efficientnet-sagemaker
)
tag = "latest"  # Example: latest


In [4]:
# Uncomment and run only when docker push fails with OOM errors
#! docker system prune -af

In [5]:
! aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 570106654206.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [6]:
from sagemaker.pytorch import PyTorch

# refer https://github.com/aws/deep-learning-containers/blob/master/available_images.md#huggingface-training-containers to get the right uri's based on region
#image_uri = '763104351884.dkr.ecr.us-west-2.amazonaws.com/huggingface-pytorch-training:1.9.1-transformers4.12.3-gpu-py38-cu111-ubuntu20.04'
#image_uri = '570106654206.dkr.ecr.us-west-2.amazonaws.com/ptddp-launcher:latest'
#Using URI with logs added to DLC
image_uri = '570106654206.dkr.ecr.us-west-2.amazonaws.com/pt-ddp-custom:latest'

# configuration for running training on smdistributed Data Parallel
# this is the only line of code change required to leverage SageMaker Distributed Data Parallel
distribution = {'pytorchddp':{ 'enabled': True }}
#distribution = {"mpi":{"enabled":True, "num_of_processes_per_host":8}}
#distribution = { "smdistributed": { "dataparallel": { "enabled": True } } }


estimator = PyTorch(
    base_job_name="ptddp-mnist-test",
    source_dir="code",
    entry_point="train_ptddp_mnist.py",
    role=role,
    framework_version="1.11.0",
    py_version="py38",
    image_uri=image_uri,
    # For training with multinode distributed training, set this count. Example: 2
    instance_count=1,
    # For training with p3dn instance use - ml.p3dn.24xlarge, with p4dn instance use - ml.p4d.24xlarge
    instance_type="ml.p4d.24xlarge",
    sagemaker_session=sess,
    # Training using SMDataParallel Distributed Training Framework
    distribution=distribution,
    debugger_hook_config=False,
)

In [7]:
estimator.fit()

2022-07-01 18:05:04 Starting - Starting the training job...ProfilerReport-1656698704: InProgress
...
2022-07-01 18:05:48 Starting - Preparing the instances for training..........................................
2022-07-01 18:13:06 Downloading - Downloading input data
2022-07-01 18:13:06 Training - Downloading the training image...........................
2022-07-01 18:17:32 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
sed: can't read changehostname.c: No such file or directory
gcc: error: changehostname.c: No such file or directory
gcc: fatal error: no input files
compilation terminated.
gcc: error: changehostname.o: No such file or directory
ERROR: ld.so: object '/libchangehostname.so' from LD_PRELOAD cannot be preloaded (cannot open shared object file): ignored.
/opt/conda/lib/python3.8/site-packages/paramiko/transport.py:236: CryptographyDeprecationW